# Linear Genetic Programming Based Approach for Robotic Controllers
In this lab session, we will leverage Linear Genetic Programming (LGP) to automatically design the controller for a mobile robot. Starting from a certain initial position (S), the robot must be able to navigate inside a maze until it reaches its home base (G). The maze consists of empty cells in which the robot can navigate freely and wall cells where the robot cannot pass. A ```Maze``` object also contains information about the optimal path to the goal, which can be used to evaluate the robot's navigation. 

In [1]:
import enum
import math
import random

from robot_maze import Maze, Robot

In [2]:
cellcodes = enum.Enum('cellcodes', 'EMPTY WALL START ROUTE GOAL')

maze_list = [
        [cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.EMPTY, cellcodes.START, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.WALL, cellcodes.EMPTY, cellcodes.EMPTY, cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL],
        [cellcodes.ROUTE, cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.EMPTY, cellcodes.EMPTY],
        [cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL],
        [cellcodes.WALL, cellcodes.ROUTE, cellcodes.EMPTY, cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE],
        [cellcodes.EMPTY, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.ROUTE, cellcodes.WALL, cellcodes.EMPTY, cellcodes.WALL, cellcodes.ROUTE],
        [cellcodes.WALL, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.ROUTE, cellcodes.WALL, cellcodes.WALL, cellcodes.WALL, cellcodes.GOAL],
    ]

maze = Maze(maze_list, cellcodes)

print(maze)

            #     #     #  
#     #  #  #     S     #  
#        #  .  .  .     #  
.  .  .  #  .  #  #     #  
.  #  .  .  .  #  #        
.  .  #  #  #  #  #  #  #  
#  .     #  .  .  .  .  .  
   .  #  #  .  #     #  .  
#  .  .  .  .  #  #  #  G  



The robot can move in 3 possible ways:

- Move forward;
- Turn left;
- Turn right;

It can perceive the environment through 6 sensors that, when activated (value 1), indicate the presence of a wall in the corresponding position.

![Robot's sensors](../img/sensors.png "Robot's sensors")

Our controller must map the boolean values coming from the sensors into an action for the robot to take.

Now it's time to code! Define a set of operators which can be included in the program that controls the robot actions and write the code to generate a random program.

In [3]:
movecodes = enum.Enum('movecodes', 'FORWARD LEFT RIGHT')
opcodes = enum.Enum('opcodes', 'NOT AND OR IF NOP')

In [4]:
def random_program(n, move_p=0.3):
  prg = []
  moves = list(movecodes)
  func = list(opcodes)
  for _ in range(0, n):
    if random.random() < move_p:
      op = random.choice(moves)
    else:
      op = random.choice(func)
    prg.append(op)
  return prg

In [5]:
rp = random_program(10)

Complete the ```Robot``` class into the ```utilities/robot_maze.py``` file with a proper ```eval``` function.

In [6]:
r = Robot(rp, maze, maxMoves=70, movecodes=movecodes, opcodes=opcodes)

Define a fitness function for our navigation task. Hint: you can exploit the ```getRoute``` method of the ```Robot``` class and the ```scoreRoute``` method of the ```Maze``` class to get an estimate of the "correct steps" taken by the robot.

In [7]:
def fit(prg):
  try:
    robot = Robot(prg, maze, 70, movecodes, opcodes)
    robot.run()
    fitness = maze.scoreRoute(robot.getRoute()) - robot.n_moves
    return fitness
  except Exception:
    return -math.inf

Implement a function to perform tournament selection

In [8]:
def tournament_selection(fit, pop, t_size=4):
  tournament = random.choices(pop, k=t_size)
  return max(tournament, key=fit)

Implement functions for crossover and mutation.

In [9]:
def crossover(x, y):
  k1 = random.randint(0, len(x)-1)
  k2 = random.randint(k1, len(x)-1)
  h1 = random.randint(0, len(y)-1)
  h2 = random.randint(h1, len(y)-1)
  of1 = x[0:k1] + y[h1:h2] + x[k2:]
  of2 = y[0:h1] + x[k1:k2] + y[h2:]
  return of1, of2

In [10]:
def mutation(x, p_m, move_p=0.3):
  def change(b):
    moves = list(movecodes)
    func = list(opcodes)
    if random.random() < p_m:
      if random.random() < move_p:
        op = random.choice(moves)
      else:
        op = random.choice(func)
      return op
    else:
      return b
  return [change(b) for b in x]

Implement a ```linear_GP``` function using the functions defined above.

In [11]:
def linear_GP(fit, pop_size, n_iter = 50):
  p_m = 0.1
  pop = [random_program(20) for _ in range(0, pop_size)]
  best = []
  for i in range(0, n_iter):
    selected = [tournament_selection(fit, pop) for _ in range(0, pop_size)]
    pairs = zip(selected, selected[1:] + [selected[0]])
    offsprings = []
    for x, y in pairs:
      of1, of2 = crossover(x, y)
      offsprings.append(of1)
      offsprings.append(of2)
    pop = [mutation(x, p_m) for x in offsprings]
    candidate_best = max(pop, key=fit)
    if fit(candidate_best) > fit(best):
      best = candidate_best
    # print(f"Best individual at generation {i}: {best}")
    print(f"Best fitness at generation {i}: {fit(best)}")
  return best

In [12]:
random.seed(0)
best = linear_GP(fit, 1000)

Best fitness at generation 0: -67
Best fitness at generation 1: -67
Best fitness at generation 2: -66
Best fitness at generation 3: -65
Best fitness at generation 4: -65
Best fitness at generation 5: -65
Best fitness at generation 6: -65
Best fitness at generation 7: -65
Best fitness at generation 8: -65
Best fitness at generation 9: -65
Best fitness at generation 10: -65
Best fitness at generation 11: -65
Best fitness at generation 12: -65
Best fitness at generation 13: -65
Best fitness at generation 14: -65
Best fitness at generation 15: -65
Best fitness at generation 16: -65
Best fitness at generation 17: -65
Best fitness at generation 18: -65
Best fitness at generation 19: -65
Best fitness at generation 20: -65
Best fitness at generation 21: -65
Best fitness at generation 22: -65
Best fitness at generation 23: -65
Best fitness at generation 24: -17
Best fitness at generation 25: -17
Best fitness at generation 26: -17
Best fitness at generation 27: -17
Best fitness at generation 28:

In [13]:
best

[<opcodes.AND: 2>,
 <opcodes.IF: 4>,
 <movecodes.FORWARD: 1>,
 <opcodes.NOT: 1>,
 <opcodes.AND: 2>,
 <opcodes.IF: 4>,
 <movecodes.LEFT: 2>,
 <opcodes.IF: 4>,
 <opcodes.IF: 4>,
 <opcodes.IF: 4>,
 <movecodes.RIGHT: 3>,
 <movecodes.FORWARD: 1>,
 <opcodes.IF: 4>,
 <opcodes.OR: 3>,
 <opcodes.AND: 2>,
 <opcodes.IF: 4>,
 <opcodes.NOP: 5>,
 <opcodes.AND: 2>,
 <opcodes.AND: 2>,
 <opcodes.IF: 4>,
 <movecodes.LEFT: 2>,
 <opcodes.NOP: 5>,
 <opcodes.OR: 3>,
 <opcodes.OR: 3>,
 <opcodes.IF: 4>,
 <opcodes.AND: 2>,
 <opcodes.AND: 2>,
 <opcodes.IF: 4>,
 <opcodes.IF: 4>,
 <movecodes.RIGHT: 3>,
 <opcodes.NOT: 1>,
 <movecodes.FORWARD: 1>]

In [14]:
best_robot = Robot(best, maze, 70, movecodes, opcodes)
best_robot.run()
print(best_robot.getRoute())

[[6, 1], [6, 2], [7, 2], [6, 2], [5, 2], [4, 2], [4, 3], [4, 4], [3, 4], [2, 4], [2, 3], [1, 3], [0, 3], [0, 4], [0, 5], [1, 5], [1, 6], [1, 7], [1, 8], [2, 8], [3, 8], [4, 8], [4, 7], [4, 6], [5, 6], [6, 6], [7, 6], [8, 6], [8, 7], [8, 8]]


Are the robot moves consistent with the expected behaviour? (The robot starts with a south heading, hence towards the bottom of the monitor)

In [15]:
print(maze)
best_robot.moves

            #     #     #  
#     #  #  #     S     #  
#        #  .  .  .     #  
.  .  .  #  .  #  #     #  
.  #  .  .  .  #  #        
.  .  #  #  #  #  #  #  #  
#  .     #  .  .  .  .  .  
   .  #  #  .  #     #  .  
#  .  .  .  .  #  #  #  G  



[<movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.RIGHT: 3>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.RIGHT: 3>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.RIGHT: 3>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.LEFT: 2>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.RIGHT: 3>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.FORWARD: 1>,
 <movecodes.RIGHT: 3>,
 <movecodes.